In [1]:
import sqlite3
import requests
import string

In [2]:
#API root and auth
sale_id_api = 'https://logistics-api-dot-active-thunder-329100.rj.r.appspot.com'
chip_api = 'https://telecom-api-dot-active-thunder-329100.rj.r.appspot.com'
conversations_api = 'https://chats-api-dot-active-thunder-329100.rj.r.appspot.com'
header = {'authorization': 'teste'} #auth code

#HTTP Requests
#   Sale API
HTTP_request_sale_track = '/tracking'
HTTP_request_sale_zip = '/zip_code'
#   Chip Api
HTTP_request_chip = '/chip_status'
#   Conversation API
HTTP_request_conversation = '/conversations'
HTTP_request_conversation_info = '/conversation_info'

In [3]:
#get for the first act conversation /!\ need to do it for all act convers
def get_sale_id_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT id_sale
        FROM sales
        WHERE sales.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    sale_id = c.fetchone()
    return {"id_sale": str(sale_id[0])} #gives a tupple, with a second null arg

In [4]:
def generate_sale_tracking_info(params_request):
    contact_info = 'customer_service@help.br'
    url = sale_id_api+HTTP_request_sale_track

    try:
        response = requests.post(url, json=params_request, headers=header)
        response.raise_for_status()
        
    except requests.exceptions.HTTPError as errh:
        return f"An Http Error occurred: Please, contact us at: {contact_info}"
    except requests.exceptions.ConnectionError as errc:
        return f"A Connection Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.Timeout as errt:
        return f"A Timeout Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.RequestException as err:
        return f"An Unknown Error occurred: Please, contact us at: {contact_info}"       
    
    return response.json()

In [5]:
def generate_sale_zip_info_v2(params_request):
    contact_info = 'customer_service@help.br'
    url = sale_id_api+HTTP_request_sale_zip 

    try:
        response = requests.post(url, json=params_request, headers=header)
        response.raise_for_status()
        
    except requests.exceptions.HTTPError as errh:
        return f"An Http Error occurred: Please, contact us at: {contact_info}"
    except requests.exceptions.ConnectionError as errc:
        return f"A Connection Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.Timeout as errt:
        return f"A Timeout Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.RequestException as err:
        return f"An Unknown Error occurred: Please, contact us at: {contact_info}"
    
    return response.json()

In [6]:
def generate_chip_info(params_request):
    contact_info = 'customer_service@help.br'
    url = chip_api+HTTP_request_chip
    
    try:
        response = requests.post(url, json=params_request, headers=header)
        response.raise_for_status()
        
    except requests.exceptions.HTTPError as errh:
        return f"An Http Error occurred: Please, contact us at: {contact_info}"
    except requests.exceptions.ConnectionError as errc:
        return f"A Connection Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.Timeout as errt:
        return f"A Timeout Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.RequestException as err:
        return f"An Unknown Error occurred: Please, contact us at: {contact_info}"
    
    return response.json()

In [7]:
#all convs info list
def generate_all_act_conv_info():
    url = conversations_api+HTTP_request_conversation
    response = requests.post(url, headers=header)
    json_response = response.json() #retrieve active conversations ID

    active_chat_list = [] #Create a list of active conversations info
    for values in json_response.values():
        for val in values:
            params_conver_info = {'conversation_id': str(val)}
            response = requests.post(conversations_api+HTTP_request_conversation_info, headers=header, json=params_conver_info)
            active_chat_list.append(response.json())

        return active_chat_list

In [8]:
#active conv list
def generate_act_convs_list():
    url = conversations_api+HTTP_request_conversation
    response = requests.post(url, headers=header)
    json_response = response.json() #retrieve active conversations ID
    
    active_convs_list = json_response.get('conversation_ids')
    return active_convs_list

In [9]:
def generate_act_conv_info(conversation_id):
    
    url = conversations_api+HTTP_request_conversation_info
    params = {"conversation_id": conversation_id}
    response = requests.post(url, json=params, headers=header)
    json_response = response.json()
    
    return json_response

In [10]:
def generate_info_of_conv(conversation_id, info_needed):
    
    conversation_id = str(conversation_id)
    info_needed = str(info_needed)
    
    url = conversations_api+HTTP_request_conversation_info
    params = {"conversation_id": conversation_id}
    response = requests.post(url, json=params, headers=header)
    json_response = response.json()

    info = json_response.get(info_needed)
    
    return info

In [11]:
generate

NameError: name 'generate' is not defined

In [ ]:
generate_all_act_conv_info()

In [ ]:
generate_act_convs_list()

In [ ]:
generate_act_conv_info('754895')

In [ ]:
get_sale_id_SQL('395023')

In [ ]:
generate_sale_tracking_info({'id_sale': '395023'})

In [ ]:
generate_sale_zip_info_v2({'zip_code': 'NA'})

In [ ]:
def delivery_api_status(param, code):
    
    code = str(code)
    param = str(param)
    
    if code == 'zip_code':
        response = generate_sale_zip_info_v2({code: param})
    elif code == 'id_sale':
        response = generate_sale_tracking_info({code: param})
    
    if isinstance(response,dict):
            
        return 'ok'
        
    else:
        return 'error'

In [ ]:
delivery_api_status({'zip_code': '02219001'},'zip_code')

In [ ]:
def responses_pipeline():
    active_chats = generate_act_convs_list()
    subject_list = []
#     while len(active_chats) > 0:
    for chat in list(active_chats): #parse all the active chats
        #parse all the active messages in a conversation
        messages = generate_info_of_conv(chat, 'messages')
        for message in list(messages):

            #generate the subject of the problem and give it to the bot AI
            subject = generate_info_of_conv(chat,'subject')
            subject_list.append(subject)
            messages.remove(message)
            
        
        active_chats.remove(chat)
    return subject_list

In [ ]:
subject_list = responses_pipeline()
for subject in subject_list:
    print(subject)

In [ ]:
def delivery_response_v2(merchant_id):
    contact_info = 'customer_service@help.br'
    #TBD check that all api are [200] if not -> answer internar error 
    
    #get id sale from SQL
    id_sale = get_sale_id_SQL(merchant_id)
    
    #---------api_sale_status--------------------
    sale_api_status = delivery_api_status(id_sale,'id_sale')
    
    if sale_api_status == 'error':
        return f'Oh, we got an internal issue please contact: {contact_info}'
    #--------------------------------
    
    
    #get info from tracking API
    tracking_info = generate_sale_tracking_info(id_sale)
    #get info zip code API
    zip_code = tracking_info.get("destination_zip_code")
    
    # /!\ if tracking_info.get("destination_zip_code") == 'NA' -> 'we missing your delivery info' -> send delivery info to api
     
    #---------api_zip_code_status--------------------
    zip_code_api_status = delivery_api_status(zip_code,'zip_code')
    
    if zip_code_api_status == 'error':
        return f'Oh, we got an internal issue please contact: {contact_info}'
    #--------------------------------
    
    
    
    
    
    zip_code_info = generate_sale_zip_info({'zip_code':zip_code})
    #variables
    delivery_forecast = tracking_info.get('delivery_forecast')
    order_status = tracking_info.get('status')
    neighborhood = zip_code_info.get('neighborhood')
    city = zip_code_info.get('city')
    street = zip_code_info.get('street')
    state = zip_code_info.get('state')
    complement = zip_code_info.get('complement')

    if complement is not None and isinstance(complement,str) and len(complement)>0: #do something when there is a complement
        return print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m, \033[1m{complement}\033[0m.\nStatus: \033[1m{order_status}\033[0m")
    else:
        return print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m.\nStatus: \033[1m{order_status}\033[0m")


In [ ]:
def delivery_response(merchant_id):
    contact_info = 'customer_service@help.br'
    #TBD check that all api are [200] if not -> answer internar error 
    
    #get id sale from SQL
    id_sale = get_sale_id_SQL(merchant_id)
    
    #---------api_sale_status--------------------
    sale_api_status = delivery_api_status(id_sale,'id_sale')
    
    if sale_api_status == 'error':
        return f'Oh, we got an internal issue please contact: {contact_info}'
    #--------------------------------
    
    
    #get info from tracking API
    tracking_info = generate_sale_tracking_info(id_sale)
    #get info zip code API
    zip_code = tracking_info.get("destination_zip_code")
    
    # /!\ if tracking_info.get("destination_zip_code") == 'NA' -> 'we missing your delivery info' -> send delivery info to api
     
    #---------api_zip_code_status--------------------
    zip_code_api_status = delivery_api_status(zip_code,'zip_code')
    
    if zip_code_api_status == 'error':
        return f'Oh, we got an internal issue please contact: {contact_info}'
    #--------------------------------
    
    
    
    
    
    zip_code_info = generate_sale_zip_info({'zip_code':zip_code})
    #variables
    delivery_forecast = tracking_info.get('delivery_forecast')
    order_status = tracking_info.get('status')
    neighborhood = zip_code_info.get('neighborhood')
    city = zip_code_info.get('city')
    street = zip_code_info.get('street')
    state = zip_code_info.get('state')
    complement = zip_code_info.get('complement')

    if complement is not None and isinstance(complement,str) and len(complement)>0: #do something when there is a complement
        return print(f"Concerning your current issue\nYour machine is expected to be delivered the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m, \033[1m{complement}\033[0m.\nStatus: \033[1m{order_status}\033[0m")
    else:
        return print(f"Concerning your current issue\nYour machine is expected to be delivered the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m.\nStatus: \033[1m{order_status}\033[0m")


In [ ]:
delivery_response('384923')

## pipeline dict with merchant id


In [14]:
# need the pipeline to be a dictionary with [conv_id, merchant_id and the subject]

In [50]:
def response_pipeline_v2():
    active_chats = generate_act_convs_list()
    list_of_problems = []

    for conversation_id in list(active_chats):

        #give info for every all messages
        messages = generate_info_of_conv(conversation_id, 'messages') #messages of one conversation

        for message in list(messages):


            merchant_id = generate_info_of_conv(conversation_id,'merchant_id')
            subject = generate_info_of_conv(conversation_id, 'subject')

            features_pipeline = {'conversation_id': conversation_id, 'merchant_id': merchant_id, 'subject': subject}

            #create a list to parse on chatbot.py
            list_of_problems.append(features_pipeline)
            messages.remove(message)

        # {'conversation_id': chat_id, 'merchant_id':}
    return list_of_problems

In [52]:
list_of_all_active_messages = response_pipeline_v2()

In [57]:
for message in list_of_all_active_messages:
    
    merchant_id = message.get('merchant_id')
    subject = message.get('subject')
    conversation_id = message..get('conversation_id')
    print(message)

{'conversation_id': 754893, 'merchant_id': '558392', 'subject': 'bank account receipt problem'}
{'conversation_id': 754894, 'merchant_id': '530364', 'subject': 'connection problem'}
{'conversation_id': 754895, 'merchant_id': '384923', 'subject': 'late delivery'}
{'conversation_id': 754896, 'merchant_id': '395023', 'subject': 'check delivery address'}
{'conversation_id': 754897, 'merchant_id': '849583', 'subject': 'delivery forecast'}


In [12]:
def responses_pipeline():
    active_chats = generate_act_convs_list()
    subject_list = []
#     while len(active_chats) > 0:
    for chat in list(active_chats): #parse all the active chats
        #parse all the active messages in a conversation
        messages = generate_info_of_conv(chat, 'messages')
        for message in list(messages):

            #generate the subject of the problem and give it to the bot AI
            subject = generate_info_of_conv(chat,'subject')
            subject_list.append(subject)
            messages.remove(message)
            
        
        active_chats.remove(chat)
    return subject_list

In [13]:
subject_list = responses_pipeline()
for subject in subject_list:
    print(subject)

bank account receipt problem
connection problem
late delivery
check delivery address
delivery forecast
